# Aula 10 - RAGAS

Leandro Carísio Fernandes

## Setup

In [51]:
DIR_AULA10 = "/content/drive/My Drive/IA024A-Redes_Neurais_NLP/Aula10-RAGAS/"

ARQUIVO_PARA_TESTAR = f'{DIR_AULA10}test_questions.json'

GERAR_FAITHFULNESS = False
GERAR_ANSWER_RELEVANCE = False
GERAR_CONTEXT_RELEVANCE = False

ARQUIVO_AVALIACAO_RAGAS = f'{DIR_AULA10}avaliacao_ragas.pickle'

In [17]:
from getpass import getpass
GROQ_API = getpass("API groq")

API groq··········


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
!pip install Groq
!pip install unidecode
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

# Download da base de dados

Para esse caderno, vamos precisar apenas do arquivo de teste fornecido pelo professor.

In [21]:
!cp "{ARQUIVO_PARA_TESTAR}" "./test_questions.json"

In [22]:
import json

test_set = json.load(open('test_questions.json', 'r'))

## Entendendo o dataset

In [23]:
i = 0 # i = 3 # Dá pra testar isso com qualquer tipo de resposta span. Para outros tipos, tem que adaptar
print('Primeira pergunta do conjunto de teste:')
print('Pergunta:', test_set[i]['question'])
print('Resposta:', test_set[i]['answer']['answer_spans'][0]['text']) # Isso só funciona pois já sabemos que é do tipo span
print('De onde veio a resposta:', test_set[i]['answer']['answer_spans'][0]['passage'])
print('---------------------')

Primeira pergunta do conjunto de teste:
Pergunta: What is Zeus know for in Greek mythology?
Resposta: sky and thunder god
De onde veio a resposta: zeus
---------------------


In [27]:
# Imprime o contexto
t = test_set[10]
contexto = [c['text'] for c in t['context']]
print('. '.join(contexto))

and studied medicine at the University of Genoa in Italy..  It is located in the city of Genoa and regional Metropolitan City of Genoa, on the Italian Riviera in the Liguria region of northwestern Italy.


##Organizando os dados

Organizando o conjunto de teste para gerar um array de pergunta/resposta/contexto.

In [192]:
perg_resp_contexto = []

for pr in test_set:
  pergunta = pr['question']
  resposta_obj = pr['answer']
  tipo_resposta = resposta_obj['type']

  if tipo_resposta == 'binary':
    resposta = resposta_obj['answer_value']
  elif tipo_resposta == 'value':
    resposta = resposta_obj['answer_value'] + ' ' + resposta_obj['answer_unit']
  elif tipo_resposta == 'span':
    resposta = resposta_obj['answer_spans'][0]['text']
  elif tipo_resposta == 'none':
    resposta = 'none'
  else:
    resposta = 'Não pode chegar aqui, os tipos parece que são só binary/value/span/none!'
    print(tipo_resposta)

  contexto = [c['text'] for c in pr['context']]
  perg_resp_contexto.append({
      "pergunta": pergunta,
      "resposta": resposta.strip(),
      "contexto": contexto # Aqui está no formato de array. Depois tem que juntar para formar uma string.
  })

  # Printa as respostas:
  print(resposta)

sky and thunder god
5 years
30 years
2 years
26,688
White Hart Lane
Germany 
Kulm
9 years
, Friedrich Max Müller,
Liguria
Cuba
1909
2000
1 year
31 
USA 
Bill de Blasio
11 Years
Herald Sun
Iraq
1897 years
Costa Rica
Colorado Buffaloes
- Frank McPhee, Princeton
- Bernie Flowers, Purdue
- Eddie Bell, Pennsylvania
- Tom Stolhandske, Texas
- Tom Scott, Virginia
- Joe Collier, Northwestern
- Don Branby, Colorado
- Buck Martin, Georgia Tech
- Steve Mellinger, Kentucky
- Ed Luke, Michigan State
- Harry Babcock, Georgia

106 years
69 years
7 May 1946
Pulp Fiction
David Austin
Cardinals 
60 years
Rochester, New York,
Sam Chase
Georgia
5 appearances
1964 
George Pesut
yes
yes
 Guaranteed Rate Field
20 years
10 years
marathon
142.8 seconds
1945
1947
no
 Donelson
United States


## Avaliação RAGAS

In [193]:
import os.path
import pandas as pd

def salvar_df_avaliacao():
  df_avaliacao.to_pickle(ARQUIVO_AVALIACAO_RAGAS)

if os.path.isfile(ARQUIVO_AVALIACAO_RAGAS):
  df_avaliacao = pd.read_pickle(ARQUIVO_AVALIACAO_RAGAS)
else:
  df_avaliacao = pd.DataFrame(perg_resp_contexto)
  df_avaliacao['faithfulness'] = ''
  df_avaliacao['answer_relevance'] = ''
  df_avaliacao['context_relevance'] = ''
  salvar_df_avaliacao()

## Faithfulness

Como para esse dataset todas as respostas são bem curtas, podemos pular a parte do método de separar a resposta em sentenças, pois a resposta do dataset já é uma sentença única.

Com isso, podemos adaptar ligeiramente o prompt proposto para essa métrica, já que não será necessário passar todas as frases.

Obs.: Como temos só uma frase para avaliar, na prática a métrica é 0 ou 1. Ou ela está no contexto ou não está.

In [194]:
system_message_faithfulness = """
Consider the given context and following statement, then determine whether it is supported by the information present in the context.

The user will provide the context and statement in JSON format and you should also answer in JSON format with two attributes: "reason" and "verdict".

Your answer should provide a brief explanation for the statement before arriving at the verdict. The veredict should be the string "Yes" or the string "No".

Do not deviate from the specified format.
""".strip()

content_message_faithfulness = """
{{
  "context": "{context}",
  "statement": "{answer}"
}}
""".strip()

In [195]:
from groq import Groq

client = Groq(api_key=GROQ_API)

def get_faithfulneess_message(context, answer):
  messages=[
    {
        "role": "system",
        "content": system_message_faithfulness
    },
    {
        "role": "user",
        "content": content_message_faithfulness.format(context=context, answer=answer)
    }
  ]
  completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=messages,
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
  )
  return completion.choices[0].message.content

In [196]:
from tqdm import tqdm

for idx, row in tqdm(df_avaliacao.iterrows(), total=len(df_avaliacao), desc="Calculando Faithfulness"):
  gerar_para_pergunta = GERAR_FAITHFULNESS or row['faithfulness'] == ''

  if gerar_para_pergunta:
    df_avaliacao.at[idx, 'faithfulness'] = get_faithfulneess_message('. '.join(row['contexto']), row['resposta'])
    salvar_df_avaliacao()



Calculando Faithfulness: 100%|██████████| 50/50 [00:00<00:00, 5552.87it/s]


In [197]:
import json

total_yes = 0
for _, row in tqdm(df_avaliacao.iterrows(), total=len(df_avaliacao), desc="Calculando Faithfulness"):
  json_msg_faithfulness = row['faithfulness']
  msg_faithfulness = json.loads(json_msg_faithfulness)

  total_yes += 1 if msg_faithfulness['verdict'] == 'Yes' else 0

print(f'\nFaithfulness: {total_yes/len(df_avaliacao):.2f}')

Calculando Faithfulness: 100%|██████████| 50/50 [00:00<00:00, 6072.01it/s]


Faithfulness: 0.68


## Answer relevance

In [198]:
system_message_answer_relevance = """
The user will provide an answer and your task is to generate a possible question for that answer.

You should answer in JSON format with two attributes: "reason" and "question".

Do not deviate from the specified format.
""".strip()

content_message_answer_relevance = """
Generate a question for the given answer.

Answer: {answer}
""".strip()

In [199]:
from groq import Groq

client = Groq(api_key=GROQ_API)

def get_answer_relevance_message(answer):
  messages=[
    {
        "role": "system",
        "content": system_message_answer_relevance
    },
    {
        "role": "user",
        "content": content_message_answer_relevance.format(answer=answer)
    }
  ]
  completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=messages,
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    #response_format={"type": "json_object"},
    stop=None,
  )
  return completion.choices[0].message.content

In [200]:
from tqdm import tqdm

for idx, row in tqdm(df_avaliacao.iterrows(), total=len(df_avaliacao), desc="Calculando Faithfulness"):
  gerar_para_pergunta = GERAR_ANSWER_RELEVANCE or row['answer_relevance'] == ''

  if gerar_para_pergunta:
    # Gera 10 perguntas
    perguntas_possiveis = []
    for i in range(10):
      perguntas_possiveis.append(get_answer_relevance_message(row['resposta']))
    df_avaliacao.at[idx, 'answer_relevance'] = perguntas_possiveis
    salvar_df_avaliacao()


Calculando Faithfulness: 100%|██████████| 50/50 [00:00<00:00, 7515.33it/s]


In [201]:
# Extrai as perguntas

import re

regex_question = r'"question"\s*:\s*"((?:[^"]|(?<=\\)")*?)"\s*[,}]'
perguntas_geradas = []

for _, row in tqdm(df_avaliacao.iterrows(), total=len(df_avaliacao), desc="Calculando Answer Relevance"):
  lista_json_msg_answer_relevance = row['answer_relevance']

  perguntas_para_essa_resposta = []
  for json_msg_answer_relevance in lista_json_msg_answer_relevance:
    # Considerando que nem sempre o LLM trouxe json bem formatados (tive que
    # desabitar o retorno para JSON), vou pegar o conteúdo da propriedade
    # "question" com regex
    question_str_match = re.search(regex_question, json_msg_answer_relevance)
    if question_str_match:
      # O json é mal formado. As vezes a string está assim "pergunta gerada",
      # mas acontece de ter alguns problemas. Por exemplo, já chegou a ter
      # resultado assim: "pergunta gerada'.
      # Por isso, guarda apenas as perguntas que dão match no padrão pesquisado
      perguntas_para_essa_resposta.append(question_str_match.group(1))

  perguntas_geradas.append(perguntas_para_essa_resposta)

Calculando Answer Relevance: 100%|██████████| 50/50 [00:00<00:00, 3842.70it/s]


In [205]:
# Gera os embeddings e calcula a métrica

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

idx = 0
AR_todas_perguntas = 0
for _, row in tqdm(df_avaliacao.iterrows(), total=len(df_avaliacao), desc="Calculando Answer Relevance"):
  # Pergunta usada para se obter a resposta
  pergunta_referencia = row['pergunta']
  # Perguntas geradas pelo LLM
  perguntas_geradas_para_pergunta_referencia = perguntas_geradas[idx]
  # Agrupa tudo em uma única lista para gerar os embeddings de uma vez
  todas_perguntas = [pergunta_referencia] + perguntas_geradas_para_pergunta_referencia
  embeddings = model.encode(todas_perguntas)

  # Cálculo do AR para essa pergunta
  AR = 0
  for i in range(1, len(embeddings)):
    AR += cosine_similarity(np.array([embeddings[0]]), np.array([embeddings[i]]))[0][0]
  AR /= len(perguntas_geradas_para_pergunta_referencia)

  # Acumula o AR para o dataset
  AR_todas_perguntas += AR

  idx += 1

# Termina o cálculo do AR para o dataset (média)
AR_todas_perguntas /= len(df_avaliacao)

# Imprime o resultado
print(f'\nAnswer relevance: {AR_todas_perguntas:.2f}')

Calculando Answer Relevance: 100%|██████████| 50/50 [00:06<00:00,  8.00it/s]


Answer relevance: 0.25


## Context relevance

In [249]:
system_message_context_relevance = """
The user will provide a question and list of sentences that can potentially help answer the question.

Your task is to identify the relevant sentences provided.

Your answer must be in JSON format with one attribute: "relevant_sentences". This attribute must contain a list of sentences that are relevant to the question.

While extracting candidate sentences you're not allowed to make any changes to the sentences provided.

Do not deviate from the specified format.
""".strip()

def get_content_message_context_relevance(question, context):
  msg = f"Question: {question}\n\nSentences:\n"
  for i, sentence in enumerate(context):
    msg = msg + f"{i} - {sentence}\n"

  return msg

In [252]:
from groq import Groq

client = Groq(api_key=GROQ_API)

def get_context_relevance_message(question, context):
  messages=[
    {
        "role": "system",
        "content": system_message_context_relevance
    },
    {
        "role": "user",
        "content": get_content_message_context_relevance(question, context)
    }
  ]
  completion = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=messages,
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    #response_format={"type": "json_object"},
    stop=None,
  )
  return completion.choices[0].message.content


In [253]:
from tqdm import tqdm

for idx, row in tqdm(df_avaliacao.iterrows(), total=len(df_avaliacao), desc="Calculando Context Relevance"):
  gerar_para_pergunta = GERAR_CONTEXT_RELEVANCE or row['context_relevance'] == ''

  if gerar_para_pergunta:
    df_avaliacao.at[idx, 'context_relevance'] = get_context_relevance_message(row['pergunta'], row['contexto'])
    salvar_df_avaliacao()

Calculando Context Relevance: 100%|██████████| 50/50 [00:44<00:00,  1.11it/s]


In [287]:
# Extrai a lista gerada
contexto = []
sentencas_relevantes = []

for _, row in tqdm(df_avaliacao.iterrows(), total=len(df_avaliacao), desc="Calculando Context Relevance"):
  contexto.append(row['contexto'])

  resposta_llm_context_relevance = row['context_relevance']
  inicio = resposta_llm_context_relevance.index('[')
  fim = resposta_llm_context_relevance.rindex(']') + 1
  str_lista = resposta_llm_context_relevance[inicio:fim]

  # Conserta algumas questões do JSON de retorno (que não veio relacionado ao padrão)
  for i in range(10):
    # Em alguns casos a informação do número da sentença veio fora da lista. Apaga isso
    str_lista = str_lista.replace(f"{i} - ", "")
  # Em alguns casos, os elementos da lista veio separado apenas por uma quebra.
  # Precisa colocar vírgula entre os termos
  str_lista = str_lista.replace("\"\n\"", "\",\n\"")

  sentencas_relevantes.append(json.loads(str_lista))

Calculando Context Relevance: 100%|██████████| 50/50 [00:00<00:00, 5150.43it/s]


In [291]:
CR = [len(sentencas_relevantes[i]) / len(contexto[i]) for i in range(len(contexto))]
print(f'Context relevance: {sum(CR)/len(CR):.2f}')

Context relevance: 0.65
